In [ ]:
%matplotlib inline
#這是juoyter notebook的magic word˙

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

# Steel Defect 瑕疵分割


In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
os.environ['TRIDENT_BACKEND'] = 'pytorch'

if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'

#為確保安裝最新版 
!pip uninstall tridentx -y
!pip install ../input/trident/tridentx-0.7.5-py3-none-any.whl --upgrade
import json
import copy
import numpy as np
#調用trident api
import trident as T
from trident import *
from trident.models import resnet,efficientnet,bisenet,densenet
import random

In [ ]:
import glob
import pandas as pd
imgs=glob.glob('../input/severstal-steel-defect-detection/train_images/*jpg')
print(len(imgs))

df_train=pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
print(df_train)


In [ ]:
df_images_frequency=df_train['ImageId'].value_counts()
print(df_images_frequency)


In [ ]:
palette = [(0, 0, 0),(256, 192, 0), (0, 192, 256), (128, 0, 256), (256,64,0)]

def enc2mask(enc, shape=(1600,256),fill_value=0):
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    s = enc.split()
    for i in range(len(s)//2):
        start = int(s[2*i]) - 1
        length = int(s[2*i+1])
        img[start:start+length] = int(fill_value)
    return img.reshape(shape).T


def mask2enc(mask, n=4):
    pixels = mask.T.flatten()
    encs = []
    for i in range(1,n+1):
        p = (pixels == i).astype(np.int8)
        if p.sum() == 0: encs.append('')
        else:
            p = np.concatenate([[0], p, [0]])
            runs = np.where(p[1:] != p[:-1])[0] + 1
            runs[1::2] -= runs[::2]
            encs.append(' '.join(str(x) for x in runs))
    return encs


def label2color(label_mask,palette):
    num_classes = len(palette)

    color_label= np.zeros((*label_mask.shape,3)).astype(np.int64)
    for i in range(num_classes):
        color_label[label_mask==i]=palette[i]
    return color_label

In [ ]:
df_train['mask'] = df_train.apply(lambda row: enc2mask(enc=row.EncodedPixels,fill_value=row.ClassId),axis=1)
print(df_train)

In [ ]:
for i in range(10):
    exsample_mask=df_train['mask'].iloc[i]
    print(exsample_mask.shape)
    print(exsample_mask.max())


In [ ]:
exsample_mask=df_train['mask'].iloc[3]
exsample_image=image2array('../input/severstal-steel-defect-detection/train_images/'+df_train['ImageId'].iloc[3])
print(exsample_mask.max())
display.display(array2image(exsample_mask))

is_mask=np.expand_dims(np.greater(exsample_mask,0).astype(np.float32),-1)
color_mask=label2color(exsample_mask,palette)


display.display(array2image(label2color(exsample_mask,palette)))
display.display(array2image(exsample_image))

display.display(array2image(0.5*exsample_image+0.5*(1-is_mask)*exsample_image+0.5*is_mask*color_mask))


In [ ]:
masked_dict=OrderedDict()
for im_path in imgs:
    folder,filename,ext=split_path(im_path)
    ImageId=filename+ext
    df_mask=df_train['ImageId']==ImageId
    filtered_df = df_train[df_mask]
    if len(filtered_df)==0:
        masked_dict[ImageId]=np.zeros((256,1600),dtype=np.int64)
    else:
        for index, row in filtered_df.iterrows():
            if ImageId not in masked_dict:
                masked_dict[ImageId]=row['mask']
            else:
                mask=row['mask']>0
                masked_dict[ImageId][mask]=row['mask'][mask]
print(len(masked_dict))

In [ ]:
calss4mask=df_train['ClassId']==4
class4_list = df_train[calss4mask]['ImageId'].tolist()

print(len(class4_list))

calss3mask=df_train['ClassId']==3
class3_list = df_train[calss3mask]['ImageId'].tolist()
print(len(class3_list))

calss2mask=df_train['ClassId']==2
class2_list = df_train[calss2mask]['ImageId'].tolist()
print(len(class2_list))

calss1mask=df_train['ClassId']==1
class1_list = df_train[calss1mask]['ImageId'].tolist()
print(len(class1_list))

In [ ]:
all_image_ids=list(masked_dict.keys())+class4_list*6+class2_list*25+class1_list*5
all_image_path=[os.path.join('../input/severstal-steel-defect-detection/train_images',imid) for imid in all_image_ids]

In [ ]:
# masked_dict=OrderedDict()

# for index, row in df_train.iterrows():
#     img_key='../input/severstal-steel-defect-detection/train_images/'+row['ImageId']
#     if img_key not in masked_dict:
#         masked_dict[img_key]=row['mask']
#     else:
#         mask=row['mask']>0
#         masked_dict[img_key][mask]=row['mask'][mask]
       

# print(len(masked_dict))

你如果試圖要把所有mask生成出來，很快你會超過kaggle記憶體上限造成notebook重啟，而若是一張一張存檔來調用看起來可行，可是會佔用不少

In [ ]:
class MyMaskDataset(MaskDataset):
    def __init__(self, masks, class_names=None, symbol="mask", **kwargs):
        super().__init__(masks,class_names=class_names, symbol=symbol, object_type=ObjectType.label_mask, **kwargs)
     
    def __getitem__(self, index: int):
        img_id = self.items[index]  # self.pop(index)
        if img_id in masked_dict:
            return masked_dict[img_id].astype(np.int64)
        else:
            return np.zeros((256,1600),dtype=np.int64)
        
#如果是mask非零則ok，否則只有20%機率取用
sample_filter=lambda x:x[-1].max()>0 or random.random()>=0.95

ds1=ImageDataset(all_image_path,symbol='image')
ds2=MyMaskDataset(all_image_ids,symbol='mask')

#設定調色盤
for i in range(5):
    ds2.palette[i] =palette[i]

data_provider=DataProvider(traindata=Iterator(data=ds1,label=ds2,sample_filter=sample_filter))
data_provider.paired_transform_funcs=[
    RandomRescaleCrop((256,256),scale=(0.9,1.1)),
    RandomTransformAffine(rotation_range=35, zoom_range=0.00, shift_range=0.02, shear_range=0.2, random_flip=0.15 ,border_mode='zero'),
    Resize((224,224),True)]
data_provider.image_transform_funcs=[
                     RandomAdjustGamma(gamma_range=(0.6,1.5)),
                     RandomAdjustContrast(value_range=(0.6, 1.5)),
                     RandomAdjustHue(value_range=(-0.5, 0.5)),
                     SaltPepperNoise(0.002),
                     Normalize(127.5,127.5)]
        

img_data,mask_data=data_provider.next()
print(mask_data.shape)
print(mask_data.max())



In [ ]:
%%time
data_provider.preview_images()

In [ ]:
from trident.models import efficientnet,deeplab,bisenet
backbond_net=efficientnet.EfficientNetB0(pretrained=True,input_shape=(3,224,224))
backbond=backbond_net.model
backbond.trainable=False
deeplabv3=deeplab.DeeplabV3_plus(backbond,atrous_rates=(6,12,18,24),num_filters=256,classes=5)
#deeplabv3.load_model('../input/steeldefect/Models/deeplabv3.pth')
deeplabv3.load_model('../input/steeldefect-sgment/Models/deeplabv3.pth.tar')
deeplabv3.summary()


In [ ]:
tiramisu = densenet.DenseNetFcn(blocks=(4,4,5,5,6), growth_rate=16, initial_filters=32, num_classes=5)
tiramisu.load_model('../input/steeldefect-sgment/Models/tiramisu.pth.tar')
tiramisu.summary()

In [ ]:
bisenetv2=ImageSegmentationModel(input_shape=(3,224,224), output=bisenet.BiSeNetV2(n_classes=5))
bisenetv2.load_model('../input/steeldefect-sgment/Models/bisenetv2.pth.tar')
bisenetv2.summary()

In [ ]:
#bisenetmodel.load_model('../input/steeldefect-sgment/Models/bisenetmodel.pth.tar')

In [ ]:
def draw_seg_image(training_context):
    data_feed = training_context['data_feed']
    data = training_context['train_data']
    model = training_context['current_model']
    output_data=data[data_feed['output']]
    target_data=to_numpy(data['mask'])
    input_data=to_numpy(data['image'])
    output_data=np.argmax(to_numpy(output_data),1)
    tile_images_list=[]
    input_arr = []
    target_arr=[]
    output_arr=[]
    for i in range(len(output_data)):
        input_arr.append(image_backend_adaption(data_provider.reverse_image_transform(input_data[i])))
        target_arr.append(label2color(target_data[i],palette))
        output_arr.append(label2color(output_data[i],palette))
    tile_images_list.append(input_arr)
    tile_images_list.append(target_arr)
    tile_images_list.append(output_arr)
    fig = tile_rgb_images(*tile_images_list, save_path='Results/segtile_image_{0}.png', imshow=True)
    plt.close()
        
        
    

In [ ]:
#.load_model('../input/steeldefect-sgment/Models/bisenetmodel.pth.tar')

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
cxt=get_session()

cxt.dice_fn=DiceLoss()
cxt.laplacian_kernel = torch.tensor([[-1.0, -1.0, -1.0],
                                      [-1.0, 8, -1.0],
                                      [-1.0, -1.0, -1.0]],dtype=torch.float32, device=get_device()).reshape(1, 1, 3, 3).requires_grad_(False)/8

# cxt.laplacian_kernel=repeat_elements(cxt.laplacian_kernel,11,axis=0)
# cxt.laplacian_kernel=repeat_elements(cxt.laplacian_kernel,11,axis=1).detach()
print(cxt.laplacian_kernel.shape)

to_rgb=ToRGB()

def get_boundary(mask_label):
    #mask_label=to_tensor(mask_label)
    B,C,H,W=int_shape(mask_label)
    mask_label=expand_dims(mask_label.reshape((B*C,H,W)),1)
    boundary_targets = F.conv2d(mask_label, cxt.laplacian_kernel, padding=1)
    boundary_targets = relu(boundary_targets)
    boundary_targets =boundary_targets.reshape((B,C,H*W))
    return boundary_targets

#邊界損失
def boundary_loss(output,target):
    shp=int_shape(output)
    masks_label_onehot=make_onehot(target,5,axis=1)
    mask_label_boundary=get_boundary(masks_label_onehot).detach()
    kernel = np.ones((3, 3), np.uint8)
    
#     mask_label_boundary =to_tensor(np.stack([cv2.dilate(im.astype(np.uint8), kernel, iterations=1).astype(np.float32) for im in to_numpy(mask_label_boundary)])).detach()
#     mask_label_boundary[mask_label_boundary>0.2]=1
       
    output_boundary=get_boundary(exp(output))

    boundary_dice=cxt.dice_fn(output_boundary,mask_label_boundary.detach())

    return boundary_dice

In [ ]:
def draw_seg_image(training_context):
    data_feed = training_context['data_feed']
    data = training_context['train_data']
    model = training_context['current_model']
    output_data=data[data_feed['output']]
    target_data=to_numpy(data['mask'])
    input_data=to_numpy(data['image'])
    output_data=np.argmax(to_numpy(output_data),1)
    tile_images_list=[]
    input_arr = []
    target_arr=[]
    output_arr=[]
    for i in range(len(output_data)):
        input_arr.append(image_backend_adaption(data_provider.reverse_image_transform(input_data[i])))
        target_arr.append(label2color(target_data[i],palette))
        output_arr.append(label2color(output_data[i],palette))
    tile_images_list.append(input_arr)
    tile_images_list.append(target_arr)
    tile_images_list.append(output_arr)
    fig = tile_rgb_images(*tile_images_list, save_path='Results/segtile_image_{0}.png', imshow=True)
    plt.close()
        

In [ ]:
def aux_dice_loss(segment_aux3,segment_aux4,segment_aux5_4,mask):
    shp=int_shape(segment_aux3)
    
#     logits_aux3 = segment_aux3.reshape((shp[0], shp[1], -1)).transpose(2,1).reshape((-1,shp[1]))
#     #print('logits_aux3',logits_aux3.shape)
#     logits_aux4 = segment_aux4.reshape((shp[0], shp[1], -1)).transpose(2,1).reshape((-1,shp[1]))
#     #print('logits_aux4',logits_aux4.shape)
#     logits_aux5_4 = segment_aux5_4.reshape((shp[0], shp[1], -1)).transpose(2,1).reshape((-1,shp[1]))
    target=mask#.reshape((-1)).detach()

    return (cxt.dice_fn(segment_aux3,target)+cxt.dice_fn(segment_aux4,target)+cxt.dice_fn(segment_aux5_4,target))/3

In [ ]:
deeplabv3.with_optimizer(optimizer=DiffGrad,lr=1e-3,betas=(0.9, 0.999),gradient_centralization='all')\
    .with_loss(DiceLoss(),loss_weight=2)\
    .with_loss(CrossEntropyLoss(ignore_index=0),loss_weight=0.5,start_epoch=2)\
    .with_loss(boundary_loss,start_epoch=1,loss_weight=2)\
    .with_loss(CrossEntropyLoss(ignore_index=0,enable_ohem=True,ohem_thresh=0.1),loss_weight=0.2,start_epoch=5)\
    .with_loss(IoULoss,start_epoch=8,loss_weight=0.5)\
    .with_metric(pixel_accuracy,name='pixel_accuracy',print_only=True)\
    .with_metric(iou,name='iou')\
    .with_accumulate_grads(5)\
    .with_regularizer('l2',reg_weight=1e-5)\
    .with_model_save_path('Models/deeplabv3.pth') \
    .with_learning_rate_scheduler(CosineLR(min_lr=1e-5,period=3000))\
    .trigger_when(when='on_batch_end',frequency=50,action=draw_seg_image)\
    .with_automatic_mixed_precision_training()

tiramisu.with_optimizer(optimizer=DiffGrad,lr=1e-3,betas=(0.9, 0.999),gradient_centralization='all')\
    .with_loss(DiceLoss(),loss_weight=2)\
    .with_loss(CrossEntropyLoss(ignore_index=0),loss_weight=0.5,start_epoch=2)\
    .with_loss(boundary_loss,start_epoch=1,loss_weight=2)\
    .with_loss(CrossEntropyLoss(ignore_index=0,enable_ohem=True,ohem_thresh=0.1),loss_weight=0.2,start_epoch=5)\
    .with_loss(IoULoss,start_epoch=8,loss_weight=0.5)\
    .with_metric(pixel_accuracy,name='pixel_accuracy',print_only=True)\
    .with_metric(iou,name='iou')\
    .with_accumulate_grads(5)\
    .with_regularizer('l2',reg_weight=1e-5)\
    .with_model_save_path('Models/tiramisu.pth') \
    .with_learning_rate_scheduler(CosineLR(min_lr=1e-5,period=3000))\
    .trigger_when(when='on_batch_end',frequency=50,action=draw_seg_image)\
    .with_automatic_mixed_precision_training()

bisenetv2.with_optimizer(optimizer=DiffGrad,lr=1e-3,betas=(0.9, 0.999),gradient_centralization='all')\
    .with_loss(DiceLoss(),loss_weight=2)\
    .with_loss(CrossEntropyLoss(ignore_index=0),loss_weight=0.5,start_epoch=2)\
    .with_loss(boundary_loss,start_epoch=1,loss_weight=2)\
    .with_loss(CrossEntropyLoss(ignore_index=0,enable_ohem=True,ohem_thresh=0.1),loss_weight=0.2,start_epoch=5)\
    .with_loss(IoULoss,start_epoch=8,loss_weight=0.5)\
    .with_metric(pixel_accuracy,name='pixel_accuracy',print_only=True)\
    .with_metric(iou,name='iou')\
    .with_accumulate_grads(5)\
    .with_regularizer('l2',reg_weight=1e-5)\
    .with_learning_rate_scheduler(CosineLR(min_lr=1e-5,period=3000))\
    .with_model_save_path('Models/bisenetv2.pth')\
    .trigger_when(when='on_batch_end',frequency=50,action=draw_seg_image)\
    .with_automatic_mixed_precision_training()

In [ ]:
plan=TrainingPlan()\
    .add_training_item(deeplabv3,name='deeplabv3')\
    .add_training_item(tiramisu,name='tiramisu')\
    .add_training_item(bisenetv2,name='bisenetv2')\
    .with_data_loader(data_provider)\
    .repeat_epochs(30)\
    .with_batch_size(32)\
    .print_progress_scheduling(10,unit='batch')\
    .display_loss_metric_curve_scheduling(frequency=100,unit='batch',imshow=True)\
    .save_model_scheduling(20,unit='batch')\

In [ ]:
plan.start_now()

In [ ]:
test_images=glob.glob('../input/severstal-steel-defect-detection/test_images/*.jpg')
print(test_images)